# Ngsolve solver
### Code的位置
* 在程序的package中位于ngsolve/python/krylovspace.py中
### package的调用
* from ngsolve.krylovspace import CGSolver
### Solver的parent class是LinearSolver，后续的iterator都是继承自这个class

## Linear Solver的构建
* 本质上是一个矩阵
* 核心的函数是 _SolveImpl，这里只是个抽象的成员函数，后续CGSolver改写了这个成员函数

In [23]:
from ngsolve import BaseMatrix, Preconditioner, BitArray, BaseVector
from typing import Optional, Callable, Union
from netgen.libngpy._meshing import _PushStatus, _GetStatus, _SetThreadPercentage
# BaseMatrix类
class LinearSolver(BaseMatrix):
    """Base class for linear solvers.
""" 
    name = "LinearSolver"
    def __init__(self, mat : BaseMatrix,
                 pre : Optional[Preconditioner] = None,
                 freedofs : Optional[BitArray] = None,
                 tol : float = None,
                 maxiter : int = 100,
                 atol : float = None,
                 callback : Optional[Callable[[int, float], None]] = None,
                 callback_sol : Optional[Callable[[BaseVector], None]] = None,
                 printrates : bool = False):
        super().__init__()
        if atol is None and tol is None:
            tol = 1e-12
        self.mat = mat
        assert (freedofs is None) != (pre is None) # either pre or freedofs must be given
        self.pre = pre if pre else Projector(freedofs, True)
        self.tol = tol
        self.atol = atol
        self.maxiter = maxiter
        self.callback = callback
        self.callback_sol = callback_sol
        self.printrates = printrates
        self.residuals = []
        self.iterations = 0
        
    def Solve(self, rhs : BaseVector, sol : Optional[BaseVector] = None,
              initialize : bool = True) -> BaseVector:
        self.iterations = 0
        self.residuals = []
        old_status = _GetStatus()
        _PushStatus(self.name + " Solve")
        _SetThreadPercentage(0)
        if sol is None:
            sol = rhs.CreateVector()
            initialize = True
        if initialize:
            sol[:] = 0
        self.sol = sol
        self._SolveImpl(rhs=rhs, sol=sol)
        if old_status[0] != "idle":
            _PushStatus(old_status[0])
            _SetThreadPercentage(old_status[1])
        return sol

## CGSolver继承自LinearSolver
* 参数输入应该是 mat = ..., pre = ..., freedofs = ...,
* 改写了 _SolveImpl，使用了
    * 在 $A$-内积下寻找共轭方向（方向彼此正交）。
    * 在这些方向上逐步最小化二次型 $\|A x-b\|_2^2$ 。

### _SolveImpl 的解释
* $r_0=b-A x_0: \mathrm{d}$ 存储当前的残差向量 $r_0$ 。
* $w_0=M^{-1} r_0: \mathrm{w}$ 存储预处理后的残差向量, 其中 $M$ 是预条件矩阵（可选)。
* $s$ 初始为 $w_0$, 表示搜索方向 $d_0$ 。
* $w d n$ 是 $r_0^{\top} M^{-1} r_0$, 用于计算步长和判断是否达到收玫条件。

### 预处理矩阵M应当是A的近似，self.pre则是M的逆的作用
* 这样$w_0 = M^{-1}r_0=M^{-1}b-M^{-1}A x_0$ 相当于求解 $M^{-1}A y = M^{-1}b$这个预处理的方程

更新解和残差
\begin{gathered}
x_{k+1}=x_k+\alpha_k d_k \\
r_{k+1}=r_k-\alpha_k A d_k
\end{gathered}
更新共轭系数和搜索方向：（$\beta$的分子是新的步长分子）
\begin{aligned}
& \beta_k=\frac{r_{k+1}^{\top} M^{-1} r_{k+1}}{r_k^{\top} M^{-1} r_k} \\
& d_{k+1}=r_{k+1}+\beta_k d_k
\end{aligned}

In [27]:
class CGSolver(LinearSolver):
    def __init__(self, *args,
                 conjugate : bool = False,
                 abstol : float = None,
                 maxsteps : int = None,
                 printing : bool = False,
                 **kwargs):
        if printing:
            print("WARNING: printing is deprecated, use printrates instead!")
            kwargs["printrates"] = printing
        if abstol is not None:
            print("WARNING: abstol is deprecated, use atol instead!")
            kwargs["abstol"] = abstol
        if maxsteps is not None:
            print("WARNING: maxsteps is deprecated, use maxiter instead!")
            kwargs["maxiter"] = maxsteps
        super().__init__(*args, **kwargs)

    def _SolveImpl(self, rhs : BaseVector, sol : BaseVector):
        d, w, s = [sol.CreateVector() for i in range(3)]
        conjugate = self.conjugate
        d.data = rhs - self.mat * sol
        w.data = self.pre * d
        s.data = w
        wdn = w.InnerProduct(d, conjugate=conjugate)
        if self.CheckResidual(sqrt(abs(wdn))):
            return

        while True:
            w.data = self.mat * s
            wd = wdn
            as_s = s.InnerProduct(w, conjugate=conjugate)       # 计算步长分母 
            if as_s == 0 or wd == 0: break
            alpha = wd / as_s
            sol.data += alpha * s                               # 更新解，沿着搜索方向走alpha
            d.data += (-alpha) * w                              # 更新残差

            w.data = self.pre * d

            wdn = w.InnerProduct(d, conjugate=conjugate)        # 计算新的步长分子
            if self.CheckResidual(sqrt(abs(wdn))):
                return

            beta = wdn / wd
            s *= beta
            s.data += w

CG 不需要显式构造 Arnoldi 或 Lanczos 基向量

### 

In [3]:
IO = True
class GatherData():
    def __init__(self,IO) -> None:
        self.iter = 0
        self.residue = 0
        self.IO = IO

    def collect(self,iters,residue):
        self.iter = max(self.iter,iters)
        self.residue = max(self.residue,residue)
        if self.IO:
            print("{} CG ends with iters {} and residue {}".format(LogTime(),iters,residue))
    
    def print(self):
        print('max CG iters = {} and max residue = {}'.format(self.iter,self.residue))
N_Iter_CG_Info = GatherData(IO) 

## CGSolver的使用
* CGSolver(mat = lhs_Niter.mat, pre = Pre.mat, maxiter=100, printrates=False,callback=N_Iter_CG_Info.collect)
* 输入参数
    * 第一个是一个矩阵
    * 第二个是预处理矩阵
    * callback：

Preconditioner(lhs_Niter,'multigrid')